In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Extract CSVs into DataFrames

In [2]:
# Original CSV without transformation 
us_deaths_file = "us_deaths_csv.csv"
us_deaths_df = pd.read_csv(us_deaths_file)
us_deaths_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Lat,Combined_Key,Population,Date,Case,Long,Country/Region,Province/State
0,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-22,0,-170.132,US,American Samoa
1,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-23,0,-170.132,US,American Samoa
2,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-24,0,-170.132,US,American Samoa
3,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-25,0,-170.132,US,American Samoa
4,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-26,0,-170.132,US,American Samoa


In [3]:
# Original CSV without transformation 
unemployment_file = "unemployment_insurance_weekly_claims.csv"
unemployment_df = pd.read_csv(unemployment_file, thousands=',')    # adding the "thousands" parameter recognizes the numbers correctly 
unemployment_df.head()

,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate
0,Alabama,1/4/2020,4578,12/28/2019,18523,1923741,0.96
1,Alabama,1/11/2020,3629,1/4/2020,21143,1923741,1.10
2,Alabama,1/18/2020,2483,1/11/2020,17402,1923741,0.90
3,Alabama,1/25/2020,2129,1/18/2020,18390,1923741,0.96
4,Alabama,2/1/2020,2170,1/25/2020,17284,1923741,0.90


# Transform us_deaths DataFrame

In [4]:
#Reformat dates: from string to date
us_deaths_df['Date'] = pd.to_datetime(us_deaths_df['Date'],infer_datetime_format=True)

# Create a filtered dataframe from specific columns 
us_deaths_cols = ['Province/State', 'Population', 'Date', 'Case']

# Rename columns 
us_deaths_col_df = us_deaths_df[us_deaths_cols].copy()
us_deaths_col_df = us_deaths_col_df.rename(columns={'Province/State':'State', "Case":"Death"})

# Create a filtered dataframe from specific dates
us_deaths_dates = pd.to_datetime(['1/25/20',
                    '2/1/20',
                    '2/8/20', 
                    '2/15/20', 
                    '2/22/20', 
                    '2/29/20', 
                    '3/7/20', 
                    '3/14/20', 
                    '3/21/20', 
                    '3/28/20', 
                    '4/4/20', 
                    '4/11/20'])
us_deaths_dates_df = us_deaths_col_df[us_deaths_col_df.Date.isin(us_deaths_dates)]

# Sort the dataframe by State then Date
us_deaths_dates_df.sort_values(['State', 'Date'], ascending=[True, True])

# Create a filtered dataframe grouped by Province/State and Date
us_deaths_transformed_df = us_deaths_dates_df.groupby(['State', 'Date']).agg({"Death": "sum", "Population": "sum"}).reset_index()
us_deaths_transformed_df.head(100)

,State,Date,Death,Population
0,Alabama,2020-01-25,0,4903185
1,Alabama,2020-02-01,0,4903185
2,Alabama,2020-02-08,0,4903185
3,Alabama,2020-02-15,0,4903185
4,Alabama,2020-02-22,0,4903185
...,...,...,...,...
95,Connecticut,2020-04-11,494,3565287
96,Delaware,2020-01-25,0,973764
97,Delaware,2020-02-01,0,973764
98,Delaware,2020-02-08,0,973764


# Transform unemployment DataFrame

In [5]:
# Rename the columns in the unemployment dataframe
unemployment_df = unemployment_df.rename(columns={"Filed week ended": "Filed_Week_Ended",
                                                "Initial Claims": "Initial_Claims",
                                                "Reflecting Week Ended": "Reflecting_Week_Ended",
                                                "Continued Claims": "Continued_Claims",
                                                "Covered Employment": "Covered_Employment",
                                                "Insured Unemployment Rate": "Insured_Unemployment_Rate"
                                                })

#Reformat dates: from string to date
unemployment_df['Filed_Week_Ended']=pd.to_datetime(unemployment_df['Filed_Week_Ended'])
unemployment_df['Reflecting_Week_Ended']=pd.to_datetime(unemployment_df['Reflecting_Week_Ended'])

# Create a filtered dataframe from specific dates
unemployment_dates = pd.to_datetime(['1/25/2020',
                    '2/1/2020',
                    '2/8/2020', 
                    '2/15/2020', 
                    '2/22/2020', 
                    '2/29/2020', 
                    '3/7/2020', 
                    '3/14/2020', 
                    '3/21/2020', 
                    '3/28/2020', 
                    '4/4/2020', 
                    '4/11/2020'])
unemployment_transformed_df = unemployment_df[unemployment_df.Filed_Week_Ended.isin(unemployment_dates)]

# Convert the "Filed_Week_Ended" column and the "Reflecting_Week_Ended" column to Date
# unemployment_transformed_df['Filed_Week_Ended']=pd.to_datetime(unemployment_transformed_df['Filed_Week_Ended'])
# unemployment_transformed_df['Reflecting_Week_Ended']=pd.to_datetime(unemployment_transformed_df['Reflecting_Week_Ended'])

# Sort the dataframe by State then Date
unemployment_transformed_df = unemployment_transformed_df.sort_values(['State', 'Filed_Week_Ended'], ascending=[True, True]).reset_index(drop=True)

unemployment_transformed_df.head(100)

,State,Filed_Week_Ended,Initial_Claims,Reflecting_Week_Ended,Continued_Claims,Covered_Employment,Insured_Unemployment_Rate
0,Alabama,2020-01-25,2129,2020-01-18,18390,1923741,0.96
1,Alabama,2020-02-01,2170,2020-01-25,17284,1923741,0.90
2,Alabama,2020-02-08,2176,2020-02-01,16745,1923741,0.87
3,Alabama,2020-02-15,1981,2020-02-08,16571,1923741,0.86
4,Alabama,2020-02-22,1735,2020-02-15,16059,1923741,0.83
...,...,...,...,...,...,...,...
95,Delaware,2020-04-11,13258,2020-04-04,29577,445741,6.64
96,District of Columbia,2020-01-25,534,2020-01-18,7149,578035,1.24
97,District of Columbia,2020-02-01,539,2020-01-25,7139,578035,1.24
98,District of Columbia,2020-02-08,508,2020-02-01,7156,578035,1.24


# Merge Dataframes

In [6]:
u_to_merge = unemployment_transformed_df.copy()
u_to_merge['unemployment_ID'] = unemployment_transformed_df.index
d_to_merge = us_deaths_transformed_df.copy()
d_to_merge['us_death_ID'] = us_deaths_transformed_df.index
merge_df = pd.merge(u_to_merge, d_to_merge, how='left', left_on=['State', 'Filed_Week_Ended'], right_on=['State', 'Date'])
merge_df

,State,Filed_Week_Ended,Initial_Claims,Reflecting_Week_Ended,Continued_Claims,Covered_Employment,Insured_Unemployment_Rate,unemployment_ID,Date,Death,Population,us_death_ID
0,Alabama,2020-01-25,2129,2020-01-18,18390,1923741,0.96,0,2020-01-25,0,4903185,0
1,Alabama,2020-02-01,2170,2020-01-25,17284,1923741,0.90,1,2020-02-01,0,4903185,1
2,Alabama,2020-02-08,2176,2020-02-01,16745,1923741,0.87,2,2020-02-08,0,4903185,2
3,Alabama,2020-02-15,1981,2020-02-08,16571,1923741,0.86,3,2020-02-15,0,4903185,3
4,Alabama,2020-02-22,1735,2020-02-15,16059,1923741,0.83,4,2020-02-22,0,4903185,4
...,...,...,...,...,...,...,...,...,...,...,...,...
631,Wyoming,2020-03-14,517,2020-03-07,3375,268518,1.26,631,2020-03-14,0,578759,691
632,Wyoming,2020-03-21,3653,2020-03-14,3197,268518,1.19,632,2020-03-21,0,578759,692
633,Wyoming,2020-03-28,6396,2020-03-21,4519,268518,1.68,633,2020-03-28,0,578759,693
634,Wyoming,2020-04-04,6543,2020-03-28,7790,268518,2.90,634,2020-04-04,0,578759,694


# Create database connection

In [ ]:
from config import password

user = "postgres"
host = "localhost"
port = "5432"
db = "COVID-19_db"
uri = f"postgresql://{user}:{password}@{host}:{port}/{db}"
engine = create_engine(uri)

# Load DataFrames into database

In [ ]:
unemployment_transformed_df.to_sql(name='unemployment', con=engine, if_exists='append', index=True)

In [ ]:
us_deaths_transformed_df.to_sql(name='us_death', con=engine, if_exists='append', index=True)

In [ ]:
merge_df.to_sql(name='merge', con=engine, if_exists='append', index=True)